In [1]:
pip install transformers

In [25]:
!pip install transformers rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=c0ae4633d18285feea48b5ee8c70920a5d9433a304e535759a8ba1d4dee0e4b2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [40]:
import json
from transformers import pipeline
from google.colab import files
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [41]:
# Upload the file
uploaded = files.upload()

Saving NuclearDataset.json to NuclearDataset (3).json


In [42]:
# Assume the uploaded file is named 'input.json'
input_file = list(uploaded.keys())[0]

In [43]:
# Load the dataset from the uploaded JSON file
with open(input_file, "r") as f:
    data = json.load(f)

In [44]:
# Load the pre-trained models for question generation and answering
question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-hl")
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [45]:
# Initialize the Rouge scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [46]:
output_data = []

In [47]:
# Generate questions and answers
for entry in data["NuclearDataset"]:
    text = entry["Text"]
    context = entry["Context"]

    # Generate question
    question_input = f"generate question: {context}"
    generated_question = question_generator(question_input)[0]['generated_text']

    # Generate answer
    generated_answer = qa_pipeline(question=generated_question, context=context)['answer']

    # Evaluate BLEU score for the generated question
    bleu_score = sentence_bleu([context.split()], generated_question.split())

    # Evaluate ROUGE score for the generated question
    rouge_score = scorer.score(context, generated_question)

    output_data.append({
        "Text": text,
        "Context": context,
        "Question": generated_question,
        "Answer": generated_answer,
        "BLEU_Score": bleu_score,
        "ROUGE_Score": rouge_score['rougeL'].fmeasure
    })

In [48]:
# Save to a JSON file
output_file = "output.json"
with open(output_file, "w") as f:
    json.dump(output_data, f, indent=4)

In [49]:
print(f"Output saved to {output_file}")

# Download the output file
files.download(output_file)

Output saved to output.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>